### **DATA FETCH**

In [347]:
!pip install firebase

uploading data file

In [348]:
from firebase import firebase
import json

# Initialize Firebase connection
FBconn = firebase.FirebaseApplication('https://targelhw2-default-rtdb.firebaseio.com/edited Audit Trail Dashboard.json', None)

# Path to the uploaded HW2.json file in Colab
file_path = 'edited Audit Trail Dashboard.json'

# Read the JSON file
with open(file_path, 'r') as file:
    data = json.load(file)

# Upload the data to Firebase
result = FBconn.put('/', 'edited Audit Trail Dashboard', data)

print('Data uploaded successfully!')
print(result)


Data uploaded successfully!
[{'Description': 'Close document', 'Document': 'Untitled document', 'Tab': 'N/A', 'Time': '2024-06-14 12:17:29', 'User': 'StudentB'}, {'Description': 'Tab Part Studio 1 of type PARTSTUDIO closed by StudentB', 'Document': 'Untitled document', 'Tab': 'Part Studio 1', 'Time': '2024-06-14 12:17:28', 'User': 'StudentB'}, {'Description': 'Tab Part Studio 1 of type PARTSTUDIO opened by StudentB', 'Document': 'Untitled document', 'Tab': 'Part Studio 1', 'Time': '2024-06-14 12:17:22', 'User': 'StudentB'}, {'Description': 'Open document', 'Document': 'Untitled document', 'Tab': 'N/A', 'Time': '2024-06-14 12:17:22', 'User': 'StudentB'}, {'Description': 'Close document', 'Document': 'Untitled document', 'Tab': 'N/A', 'Time': '2023-09-04 05:33:12', 'User': 'StudentA'}, {'Description': 'Update Part Metadata', 'Document': 'Untitled document', 'Tab': 'Part Studio 1', 'Time': '2023-09-04 05:33:12', 'User': 'StudentA'}, {'Description': 'Tab Part Studio 1 of type PARTSTUDIO cl

In [349]:
import requests
from bs4 import BeautifulSoup

def fetch_page(url):
    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        return soup
    else:
        return None

In [350]:
import re

def index_words(soup):
    index = {}
    words = re.findall(r'\w+', soup.get_text())
    for word in words:
        word = word.lower()
        if word in index:
            index[word] += 1
        else:
            index[word] = 1
    return index

In [351]:
def remove_stop_words(index):
    stop_words = {
        'a', 'about', 'above', 'after', 'again', 'against', 'all', 'am', 'an', 'and', 'any', 'are', 'as',
        'at', 'be', 'because', 'been', 'before', 'being', 'below', 'between', 'both', 'but', 'by',
        'can', 'could', 'did', 'do', 'does', 'doing', 'down'
    }

    for stop_word in stop_words:
        if stop_word in index:
            del index[stop_word]

    return index


In [352]:
from nltk.stem import PorterStemmer

def apply_stemming(index):
    stemmer = PorterStemmer()
    stemmed_index = {}
    for word, count in index.items():
        stemmed_word = stemmer.stem(word)
        if stemmed_word in stemmed_index:
            stemmed_index[stemmed_word] += count
        else:
            stemmed_index[stemmed_word] = count
    return stemmed_index

In [353]:
def search(query, index):

    stemmer = PorterStemmer()
    query_words = re.findall(r'\w+', query.lower())
    results = {}
    for word in query_words:
        word = stemmer.stem(word)
        if word in index:
            results[word] = index[word]
    return results

In [354]:
def search_engine(url, query):
    soup = fetch_page(url)
    if soup is None:
        return None
    index = index_words(soup)
    index = remove_stop_words(index)
    index = apply_stemming(index)
    results = search(query, index)
    return results

In [355]:
# New array to store the results
results = []
url = 'https://cad.onshape.com/help/Content/Glossary/glossary.htm?tocpath=_____19'

# Execute the search engine function for each word and store the results
query = {'point', 'dimension', 'studio', 'help', 'n/a', 'render', 'onshape', 'context', 'Add', 'Shortcut'}

for word in query:
    result = search_engine(url, word)
    results.append(result)

In [356]:
results

[{'dimens': 49},
 {'help': 13},
 {'render': 159},
 {'context': 676},
 {'shortcut': 631},
 {'n': 505},
 {'point': 74},
 {'onshap': 98},
 {'add': 26},
 {'studio': 369}]

In [357]:
!pip install requests

In [358]:
!pip install requests firebase

uploading the term and frequequency to firebase

In [359]:
import requests
import json


term_freq = {}
for index, item in enumerate(results):
    for term, freq in item.items():
        term_freq[index] = {'term': term, 'freq': freq}

firebase_url = 'https://targelhw2-default-rtdb.firebaseio.com/TermFreq.json'
response = requests.put(firebase_url, data=json.dumps(term_freq))

if response.status_code == 200:
    print("Data successfully added to Firebase")
else:
    print(f"Failed to add data to Firebase. Error: {response.text}")

Data successfully added to Firebase


# **Installation**

In [360]:
!pip install ipywidgets matplotlib squarify
!pip install wordcloud
!pip install firebase
!pip install squarify

## **Imports**

In [361]:
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
import matplotlib.pyplot as plt
import seaborn as sns
import io
import base64
from collections import Counter
import pandas as pd
import numpy as np
import requests
import json

# **Data**

In [362]:
# Firebase URL
firebase_url = 'https://targelhw2-default-rtdb.firebaseio.com/edited Audit Trail Dashboard.json'

# Function to fetch data from Firebase
def fetch_data_from_firebase():
    response = requests.get(firebase_url)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Failed to retrieve data: {response.text}")
        return None

# Fetch data from Firebase
data = fetch_data_from_firebase()

# If data is None or empty, use a fallback empty list
if not data:
    data = []

# **CSS**

In [363]:
def inject_css():
    css = """
    <style>
        .par-screen {
          max-width: 100%;
          background-color: #e6f3e6; /* Light green background */
          padding: 20px;
          border-radius: 10px;
        }
        .par-screen .widget-label {
          border: 2px dotted #f5f0e4;
          font-weight: bold;
          font-size: 12px;
          color: black;
          padding: 0 8px;
          width: 100%;
        }
        .error {
          font-size: 12px;
          color: white;
          background-color: red;
          margin: 150px auto;
          width: 25%;
          text-align: center;
        }
        .par-screen .title {
          font-size: 20px;
          text-align: center;
          border: 2px outset #d6b86f;
        }
        .par-screen .screen-title {
          font-size: 32px;
          text-align: center;
          border: 2px outset #d6b86f;
          margin: 16px auto;
          width: 70%;
          padding: 4px;
          background-color: white; /* White background for contrast */
        }
        .par-screen .component {
          border: 2px dotted #261e0c;
          padding: 4px;
          background-color: white; /* White background for contrast */
          margin-top: 10px;
        }
        .par-screen .widget-hbox {
          width: 100%;
          margin: 4px auto;
          height: 100%;
        }
        .menu-button {
          margin: 5px;
        }
        .custom-button {
          margin: 10px 0;
        }
        .large-dropdown .widget-dropdown select {
          width: 200px; /* Adjust width */
          height: 40px; /* Adjust height */
          font-size: 16px; /* Adjust font size */
        }
        .large-dropdown .widget-dropdown label {
          font-size: 16px;
        }
    </style>
    """
    display(HTML(css))

In [364]:
par_style = """
<style>
    .par-screen {
      max-width: 100%;
      background-color: #e6f3e6; /* Light green background */
      padding: 20px;
      border-radius: 10px;
    }
    .par-screen .widget-label {
      border: 2px dotted #f5f0e4;
      font-weight: bold;
      font-size: 12px;
      color: black;
      padding: 0 8px;
      width: 100%;
    }
    .error {
      font-size: 12px;
      color: white;
      background-color: red;
      margin: 150px auto;
      width: 25%;
      text-align: center;
    }
    .par-screen .title {
      font-size: 20px;
      text-align: center;
      border: 2px outset #d6b86f;
    }
    .par-screen .screen-title {
      font-size: 32px;
      text-align: center;
      border: 2px outset #d6b86f;
      margin: 16px auto;
      width: 70%;
      padding: 4px;
      background-color: white; /* White background for contrast */
    }
    .par-screen .component {
      border: 2px dotted #261e0c;
      padding: 4px;
      background-color: white; /* White background for contrast */
      margin-top: 10px;
    }
    .par-screen .widget-hbox {
      width: 100%;
      margin: 4px auto;
      height: 100%;
    }
    .menu-button {
      margin: 5px;
    }
    .custom-button {
      margin: 10px 0;
    }
</style>
"""

# **Project Updates and Results Screen**

In [365]:
def show_project_updates(b=None):
    clear_output()
    inject_css()

    data = fetch_data_from_firebase()

    html_content = """
    <div class="par-screen">
        <div class="screen-title">Project Updates</div>
        <div class="component" style="max-height: 600px; overflow-y: auto;">
            <h3>Last Updates:</h3>
            <ul>
    """

    for item in data:
        html_content += f"<li><strong>{item['User']}:</strong> {item['Description']} ({item['Time']})</li>"

    html_content += """
            </ul>
        </div>
    </div>
    """

    display(HTML(html_content))

    next_button = widgets.Button(description='Next', button_style='primary', layout=widgets.Layout(width='48%'))
    next_button.add_class('custom-button')
    next_button.on_click(lambda b: show_result_updates())

    back_button = widgets.Button(description='Back to Main Menu', button_style='', layout=widgets.Layout(width='48%'))
    back_button.add_class('custom-button')
    back_button.on_click(lambda b: show_main_menu())

    # Arrange buttons in a horizontal box
    button_box = widgets.HBox([back_button, next_button], layout=widgets.Layout(width='100%', justify_content='space-between'))

    display(button_box)

def show_result_updates(b=None):
    clear_output()
    inject_css()

    # Fetch data from Firebase
    data = fetch_data_from_firebase()

    # Get the last item from the data if available
    selected_data = data[-1] if data else {}

    html_content = f"""
    <div class="par-screen">
        <div class="screen-title">Selected Parameters</div>
        <div class="component">
            <p><strong>User:</strong> {selected_data.get('User', 'N/A')}</p>
            <p><strong>Description:</strong> {selected_data.get('Description', 'N/A')}</p>
            <p><strong>Time:</strong> {selected_data.get('Time', 'N/A')}</p>
            <p><strong>Document:</strong> {selected_data.get('Document', 'N/A')}</p>
            <p><strong>Tab:</strong> {selected_data.get('Tab', 'N/A')}</p>
        </div>
    </div>
    """

    display(HTML(html_content))

    back_button = widgets.Button(description='Back', button_style='', layout=widgets.Layout(width='48%'))
    next_button = widgets.Button(description='Next', button_style='primary', layout=widgets.Layout(width='48%'))
    back_button.add_class('custom-button')
    next_button.add_class('custom-button')

    back_button.on_click(lambda b: show_project_updates())
    next_button.on_click(lambda b: show_graphs())

    display(widgets.HBox([back_button, next_button], layout=widgets.Layout(width='100%', justify_content='space-between')))
    display(HTML("</div>"))

# **Graphs**

In [366]:
def show_graphs(b=None):
    clear_output()
    inject_css()

    # Fetch data from Firebase
    firebase_url = 'https://targelhw2-default-rtdb.firebaseio.com/TermFreq.json'
    response = requests.get(firebase_url)
    if response.status_code == 200:
        term_freq = response.json()
    else:
        print(f"Failed to fetch data from Firebase. Error: {response.text}")
        return

    # Prepare data for charts
    data = fetch_data_from_firebase()
    if not data:
        print("No data available.")
        return

    df = pd.DataFrame(data)
    df['Time'] = pd.to_datetime(df['Time'])
    df = df.sort_values('Time')

    # Changes by User (Bar Chart)
    user_counts = df['User'].value_counts()

    # Actions Over Time (Line Chart)
    actions_over_time = df.set_index('Time').resample('D').size()

    # Actions per Document (Pie Chart)
    document_counts = df['Document'].value_counts()

    # Frequency of Actions (Line Chart)
    actions_frequency = df.set_index('Time').resample('D').size()

    # Description Word Frequency (Bar Chart)
    word_freq_df = pd.DataFrame([(item['term'], item['freq']) for item in term_freq],
                                columns=['word', 'freq'])
    word_freq_df = word_freq_df.sort_values('freq', ascending=False).head(10)

    # Notifications at each timestamp (Bar Chart)
    notifications_per_timestamp = df['Time'].value_counts().sort_index()

    fig, axs = plt.subplots(3, 2, figsize=(14, 18))
    fig.patch.set_facecolor('#d0f0c0')  # Light green background for the figure

    # Bar chart for Changes by User
    axs[0, 0].bar(user_counts.index, user_counts.values, color='#4caf50')
    axs[0, 0].set_title('Changes by User', color='black')
    axs[0, 0].set_ylabel('Number of Changes', color='black')
    axs[0, 0].tick_params(colors='black')
    axs[0, 0].set_facecolor('#d0f0c0')  # Light green background for the plot
    plt.setp(axs[0, 0].get_xticklabels(), rotation=45, ha='right')

    # Line chart for Actions Over Time
    axs[0, 1].plot(actions_over_time.index, actions_over_time.values, color='#388e3c', marker='o')
    axs[0, 1].set_title('Actions Over Time', color='black')
    axs[0, 1].set_ylabel('Number of Actions', color='black')
    axs[0, 1].tick_params(colors='black')
    axs[0, 1].set_facecolor('#d0f0c0')  # Light green background for the plot

    # Pie chart for Actions per Document
    axs[1, 0].pie(document_counts.values, labels=document_counts.index, autopct='%1.1f%%', startangle=90, colors=['#4caf50', '#a5d6a7', '#81c784', '#66bb6a'])
    axs[1, 0].set_title('Actions per Document', color='black')
    axs[1, 0].set_facecolor('#d0f0c0')  # Light green background for the plot

    # Line chart for Frequency of Actions
    axs[1, 1].plot(actions_frequency.index, actions_frequency.values, color='#66bb6a', marker='o')
    axs[1, 1].set_title('Frequency of Actions', color='black')
    axs[1, 1].set_ylabel('Number of Actions', color='black')
    axs[1, 1].tick_params(colors='black')
    axs[1, 1].set_facecolor('#d0f0c0')  # Light green background for the plot

    # Bar chart for Word Frequency in Descriptions
    bars = axs[2, 0].bar(word_freq_df['word'], word_freq_df['freq'], color='#81c784')
    axs[2, 0].set_title('Word Frequency in Descriptions', color='black')
    axs[2, 0].set_xlabel('Words', color='black')
    axs[2, 0].set_ylabel('Frequency', color='black')
    axs[2, 0].tick_params(colors='black')
    axs[2, 0].set_facecolor('#d0f0c0')  # Light green background for the plot
    plt.setp(axs[2, 0].get_xticklabels(), rotation=45, ha='right')

    # Add value labels on top of bars
    for bar in bars:
        height = bar.get_height()
        axs[2, 0].text(bar.get_x() + bar.get_width()/2., height,
                f'{height}',
                ha='center', va='bottom', color='black')

    # Bar chart for Notifications at each timestamp
    axs[2, 1].bar(notifications_per_timestamp.index, notifications_per_timestamp.values, color='#4caf50')
    axs[2, 1].set_title('Notifications at Each Timestamp', color='black')
    axs[2, 1].set_xlabel('Time', color='black')
    axs[2, 1].set_ylabel('Number of Notifications', color='black')
    axs[2, 1].tick_params(colors='black')
    axs[2, 1].set_facecolor('#d0f0c0')  # Light green background for the plot
    plt.setp(axs[2, 1].get_xticklabels(), rotation=45, ha='right')

    plt.tight_layout()

    # Convert the plots to base64
    buf = io.BytesIO()
    plt.savefig(buf, format='png', facecolor='#d0f0c0')
    buf.seek(0)
    img_base64 = base64.b64encode(buf.read()).decode('utf-8')
    buf.close()

    html_content = f"""
    <div class="app-container">
        <div class="app-header">Statistics</div>
        <div class="app-content">
            <div style="text-align: center;">
                <img src="data:image/png;base64,{img_base64}" alt="Graphs" style="max-width: 80%;">
            </div>
        </div>
    </div>
    """


    back_button = widgets.Button(description='Back', button_style='', layout=widgets.Layout(width='48%'))
    home_button = widgets.Button(description='Home', button_style='', layout=widgets.Layout(width='48%'))
    back_button.add_class('custom-button')
    home_button.add_class('custom-button')

    back_button.on_click(lambda b: show_result_updates())
    home_button.on_click(lambda b: show_main_menu())

    display(widgets.HBox([back_button, home_button], layout=widgets.Layout(width='100%', justify_content='space-between')))

# **JSON File**

In [367]:
def show_admin_screen(b=None):
    clear_output()
    inject_css()

    screen_title = widgets.HTML("""
    <div class="screen-title">Admin Panel - Upload JSON File</div>
    """)

    instruction = widgets.HTML("""
    <div class="component">
        <p>Select a JSON file to upload:</p>
    </div>
    """)

    file_upload = widgets.FileUpload(
        accept='.json',
        multiple=False
    )

    upload_button = widgets.Button(description='Upload', button_style='primary', layout=widgets.Layout(width='100%'))
    upload_button.add_class('custom-button')

    output_area = widgets.Output(layout=widgets.Layout(width='100%', min_height='200px'))

    def on_upload_click(b):
        with output_area:
            output_area.clear_output()

            if file_upload.value:
                filename = list(file_upload.value.keys())[0]
                content = file_upload.value[filename]['content']
                try:
                    json_data = json.loads(content.decode('utf-8'))

                    json_display = widgets.HTML(
                        value=f'<pre>{json.dumps(json_data, indent=2)}</pre>',
                        layout=widgets.Layout(border='1px solid black', padding='10px', overflow='auto', max_height='400px')
                    )

                    display(json_display)

                except json.JSONDecodeError:
                    display(HTML("<p style='color: red;'>Error: Invalid JSON file</p>"))
            else:
                display(HTML("<p style='color: red;'>Please select a file first</p>"))

    upload_button.on_click(on_upload_click)

    back_button = widgets.Button(description='Back to Main Menu', button_style='', layout=widgets.Layout(width='100%'))
    back_button.add_class('custom-button')
    back_button.on_click(lambda b: show_main_menu())

    content = widgets.VBox([
        screen_title,
        instruction,
        file_upload,
        upload_button,
        output_area,
        back_button
    ])

    screen = widgets.VBox([content], layout=widgets.Layout(width='80%', margin='auto'))
    screen.add_class('par-screen')

    page.children = [create_menu(), screen]
    display(page)

# **On Shape Search**

In [368]:
def show_onshape_search_screen(b=None):
    clear_output()
    inject_css()

    html_content = """
    <div class="par-screen">
        <div class="screen-title">onShape Index Search</div>
        <div class="component">
            <p>Enter your search term:</p>
        </div>
    </div>
    """

    search_input = widgets.Text(
        placeholder='Enter search term',
        layout=widgets.Layout(width='80%')
    )

    search_button = widgets.Button(description='Search', button_style='primary', layout=widgets.Layout(width='80%'))
    search_button.add_class('custom-button')

    output = widgets.Output()

    def on_search_click(b):
        with output:
            clear_output()
            term = search_input.value.lower()
            if term:
                # Fetch data from Firebase
                firebase_url = 'https://targelhw2-default-rtdb.firebaseio.com/TermFreq.json'
                response = requests.get(firebase_url)
                if response.status_code == 200:
                    term_freq = response.json()
                    if term_freq and isinstance(term_freq, list):
                        found = False
                        for item in term_freq:
                            if isinstance(item, dict) and 'term' in item and 'freq' in item:
                                if item['term'].lower() == term:
                                    print(f"Term: {item['term']}")
                                    print(f"Frequency: {item['freq']}")
                                    found = True
                                    break
                        if not found:
                            print(f"No results found for '{term}'")
                    else:
                        print("No valid data available in the database")
                else:
                    print(f"Failed to fetch data from Firebase. Error: {response.text}")
            else:
                print("Please enter a search term")

    search_button.on_click(on_search_click)

    back_button = widgets.Button(description='Back to Main Menu', button_style='', layout=widgets.Layout(width='80%'))
    back_button.add_class('custom-button')
    back_button.on_click(lambda b: show_main_menu())

    content = widgets.VBox([
        widgets.HTML(html_content),
        search_input,
        search_button,
        output,
        back_button
    ])

    screen = widgets.VBox([content], layout=widgets.Layout(width='80%', margin='auto'))
    screen.add_class('par-screen')

    page.children = [create_menu(), screen]
    display(page)


# **Notification Setting**

In [369]:
def show_notification_settings(b=None):
    clear_output()
    inject_css()

    html_content = """
    <div class="par-screen">
        <div class="screen-title">Notification Settings/Personalization</div>
        <div class="component">
            <p>Customize your notification preferences:</p>
        </div>
    </div>
    """

    # Create checkboxes with increased size
    checkbox1 = widgets.Checkbox(
        value=True,
        description='Receive notifications for project updates',
        style={'description_width': 'initial'}
    )
    checkbox2 = widgets.Checkbox(
        value=False,
        description='Receive reminders for deadlines',
        style={'description_width': 'initial'}
    )
    checkbox3 = widgets.Checkbox(
        value=True,
        description='Receive feedback notifications',
        style={'description_width': 'initial'}
    )

    # Adjust the size of the checkboxes with a CSS class
    checkbox1.add_class('large-checkbox')
    checkbox2.add_class('large-checkbox')
    checkbox3.add_class('large-checkbox')

    settings_form = widgets.VBox([
        widgets.HBox([checkbox1]),
        widgets.HBox([checkbox2]),
        widgets.HBox([checkbox3])
    ])

    save_button = widgets.Button(description='Save Settings', button_style='primary', layout=widgets.Layout(width='200px'))
    save_button.add_class('custom-button')

    def on_save_click(b):
        display(HTML("<p>Settings saved successfully</p>"))

    save_button.on_click(on_save_click)

    back_button = widgets.Button(description='Back to Main Menu', button_style='', layout=widgets.Layout(width='200px'))
    back_button.add_class('custom-button')
    back_button.on_click(lambda b: show_main_menu())

    content = widgets.VBox([
        widgets.HTML(html_content),
        settings_form,
        save_button,
        back_button
    ])

    screen = widgets.VBox([content], layout=widgets.Layout(width='80%', margin='auto'))
    screen.add_class('par-screen')

    page.children = [create_menu(), screen]
    display(page)


# **Recent Settings**

In [370]:

def remove_duplicates(notifications):
    seen = set()
    unique_notifications = []
    for notif in notifications:
        notif_tuple = (notif.get('Time'), notif.get('Document'), notif.get('Tab'), notif.get('User'), notif.get('Description'))
        if notif_tuple not in seen:
            seen.add(notif_tuple)
            unique_notifications.append(notif)
    return unique_notifications

def categorize_notifications(notifications):
    categories = {}
    for notif in notifications:
        description = notif.get('Description', 'Unknown')
        if description not in categories:
            categories[description] = []
        categories[description].append(notif)
    return categories

def filter_notifications(notifications, time_filter, student_filter, description_filter):
    filtered = []
    min_date = datetime.datetime.strptime('2022-11-03', "%Y-%m-%d")
    max_date = datetime.datetime.strptime('2023-09-04', "%Y-%m-%d")
    today_date = datetime.datetime.strptime('2023-09-04', "%Y-%m-%d")  # Fixed "Today" date

    if time_filter == 'Last Month':
        # Define the date range for the last month from 4th August 2023 to 4th September 2023
        start_date = datetime.datetime.strptime('2023-08-04', "%Y-%m-%d")
        end_date = datetime.datetime.strptime('2023-09-04', "%Y-%m-%d")
    else:
        start_date = min_date
        end_date = max_date

    for notif in notifications:
        notif_time = datetime.datetime.strptime(notif.get('Time', ''), "%Y-%m-%d %H:%M:%S")

        # Filter by time
        if time_filter == 'Today':
            if notif_time.date() != today_date.date():
                continue
        elif time_filter == 'Last Week':
            if (start_date - notif_time).days > 7:
                continue
        elif time_filter == 'Last Month':
            if (start_date - notif_time).days > 30:
                continue
        elif time_filter == 'Last Year':
            if (start_date - notif_time).days > 365:
                continue
        elif time_filter == 'All':
            if not (min_date <= notif_time <= max_date):
                continue

        # Filter by student
        if student_filter and notif.get('User') != student_filter:
            continue

        # Filter by description
        if description_filter and notif.get('Description') != description_filter:
            continue

        filtered.append(notif)

    return filtered

def show_recent_notifications(b=None):
    clear_output()
    inject_css()

    notifications = fetch_data(NOTIFICATION_URL)
    if notifications and isinstance(notifications, list):
        # Remove duplicates and categorize
        notifications = remove_duplicates(notifications)
        categories = categorize_notifications(notifications)

        html_content = """
        <div class="par-screen">
            <div class="screen-title">Recent Notifications</div>
            <div class="component">
                <p>Select filters to view recent notifications:</p>
            </div>
        </div>
        """

        # Dropdown options
        time_options = ['Today', 'Last Week', 'Last Month', 'Last Year', 'All']
        student_options = ['All'] + sorted(set(notif.get('User') for notif in notifications if notif.get('User')))
        description_options = ['All'] + sorted(categories.keys())

        # Create dropdown widgets
        time_dropdown = widgets.Dropdown(
            options=time_options,
            value='All',
            description='Time:',
            style={'description_width': 'initial'}
        )

        student_dropdown = widgets.Dropdown(
            options=student_options,
            value='All',
            description='Student:',
            style={'description_width': 'initial'}
        )

        description_dropdown = widgets.Dropdown(
            options=description_options,
            value='All',
            description='Description:',
            style={'description_width': 'initial'}
        )

        # Adjust the size of the dropdowns with a CSS class
        time_dropdown.add_class('large-dropdown')
        student_dropdown.add_class('large-dropdown')
        description_dropdown.add_class('large-dropdown')

        filter_form = widgets.VBox([
            widgets.HBox([time_dropdown]),
            widgets.HBox([student_dropdown]),
            widgets.HBox([description_dropdown])
        ])

        apply_button = widgets.Button(description='Apply Filters', button_style='primary', layout=widgets.Layout(width='200px'))
        apply_button.add_class('custom-button')

        # Create an output widget to display filtered results
        output_area = widgets.Output(layout=widgets.Layout(width='100%', height='300px', overflow='auto'))

        def on_apply_click(b):
            with output_area:
                clear_output()

                # Get filter values
                time_filter = time_dropdown.value
                student_filter = student_dropdown.value if student_dropdown.value != 'All' else None
                description_filter = description_dropdown.value if description_dropdown.value != 'All' else None

                # Filter notifications based on the selected criteria
                filtered_notifications = filter_notifications(notifications, time_filter, student_filter, description_filter)

                # Show filtered notifications
                html_content = """
                <div class="component">
                    <h3>Filtered Notifications:</h3>
                    <ul>
                """

                if filtered_notifications:
                    for notification in filtered_notifications:
                        html_content += f"<li><strong>{notification.get('User', 'N/A')}:</strong> {notification.get('Description', 'N/A')} ({notification.get('Time', 'N/A')})</li>"
                else:
                    html_content += "<li>No notifications found based on the selected filters.</li>"

                html_content += "</ul></div>"

                display(HTML(html_content))

        apply_button.on_click(on_apply_click)

        back_button = widgets.Button(description='Back to Main Menu', button_style='', layout=widgets.Layout(width='200px'))
        back_button.add_class('custom-button')
        back_button.on_click(lambda b: show_main_menu())

        content = widgets.VBox([
            widgets.HTML(html_content),
            filter_form,
            apply_button,
            output_area,
            back_button
        ])

        screen = widgets.VBox([content], layout=widgets.Layout(width='80%', margin='auto'))
        screen.add_class('par-screen')

        page.children = [create_menu(), screen]
        display(page)

    else:
        html_content = """
        <div class="par-screen">
            <div class="screen-title">Recent Notifications</div>
            <div class="component">
                <p>Failed to retrieve notifications or no notifications available.</p>
            </div>
        </div>
        """

        content = widgets.HTML(html_content)

        back_button = widgets.Button(description='Back to Main Menu', button_style='', layout=widgets.Layout(width='200px'))
        back_button.add_class('custom-button')
        back_button.on_click(lambda b: show_main_menu())

        screen = widgets.VBox([content, back_button], layout=widgets.Layout(width='80%', margin='auto'))
        screen.add_class('par-screen')

        page.children = [create_menu(), screen]
        display(page)


# **Main Menu**

In [371]:
# Global variable for the main page
page = widgets.VBox(layout=widgets.Layout(width="100%", height="100%"))
page.add_class("page")

# CSS styles
par_style = """
<style>
    .par-screen {
      max-width: 100%;
      background-color: #e6f3e6; /* Light green background */
      padding: 20px;
      border-radius: 10px;
    }
    .par-screen .widget-label {
      border: 2px dotted #f5f0e4;
      font-weight: bold;
      font-size: 12px;
      color: black;
      padding: 0 8px;
      width: 100%;
    }
    .error {
      font-size: 12px;
      color: white;
      background-color: red;
      margin: 150px auto;
      width: 25%;
      text-align: center;
    }
    .par-screen .title {
      font-size: 20px;
      text-align: center;
      border: 2px outset #d6b86f;
    }
    .par-screen .screen-title {
      font-size: 32px;
      text-align: center;
      border: 2px outset #d6b86f;
      margin: 16px auto;
      width: 70%;
      padding: 4px;
      background-color: white; /* White background for contrast */
    }
    .par-screen .component {
      border: 2px dotted #261e0c;
      padding: 4px;
      background-color: white; /* White background for contrast */
      margin-top: 10px;
    }
    .par-screen .widget-hbox {
      width: 100%;
      margin: 4px auto;
      height: 100%;
    }
    .menu-button {
      margin: 5px;
    }
    .custom-button {
      margin: 10px 0;
    }
</style>
"""
def inject_css():
    display(HTML(par_style))

def show_main_menu():
    clear_output()
    inject_css()

    html_content = """
    <div class="par-screen">
        <div class="screen-title">Main Menu</div>
        <div class="component">
            <p>Select an option:</p>
        </div>
    </div>
    """

    content = widgets.HTML(html_content)

    project_updates_button = widgets.Button(description='Project Updates', button_style='', layout=widgets.Layout(width='200px'))
    admin_button = widgets.Button(description='Admin Panel', button_style='', layout=widgets.Layout(width='200px'))
    onshape_search_button = widgets.Button(description='onShape Search', button_style='', layout=widgets.Layout(width='200px'))
    notification_settings_button = widgets.Button(description='Notification Settings', button_style='', layout=widgets.Layout(width='200px'))
    recent_notifications_button = widgets.Button(description='Recent Notifications', button_style='', layout=widgets.Layout(width='200px'))

    buttons = [project_updates_button, admin_button, onshape_search_button, notification_settings_button, recent_notifications_button]

    for button in buttons:
        button.add_class('custom-button')

    project_updates_button.on_click(show_project_updates)
    admin_button.on_click(show_admin_screen)
    onshape_search_button.on_click(show_onshape_search_screen)
    notification_settings_button.on_click(show_notification_settings)
    recent_notifications_button.on_click(show_recent_notifications)

    buttons_box = widgets.VBox(buttons)
    buttons_box.add_class('component')

    main_content = widgets.VBox([content, buttons_box])
    main_content.add_class('par-screen')

    page.children = [create_menu(), main_content]
    display(page)


def create_menu():
    home_button = widgets.Button(description="Home", layout=widgets.Layout(disabled=True))
    project_updates_button = widgets.Button(description="Project Updates")
    admin_button = widgets.Button(description="Admin")
    onshape_search_button = widgets.Button(description="onShape Search")
    notification_settings_button = widgets.Button(description="Notification Settings")
    recent_notifications_button = widgets.Button(description="Recent Notifications")

    buttons = [home_button, project_updates_button, admin_button, onshape_search_button,
               notification_settings_button, recent_notifications_button]

    for button in buttons:
        button.layout = widgets.Layout(height='auto', width='auto')
        button.add_class('menu-button')

    home_button.on_click(lambda b: show_main_menu())
    project_updates_button.on_click(show_project_updates)
    admin_button.on_click(show_admin_screen)
    onshape_search_button.on_click(show_onshape_search_screen)
    notification_settings_button.on_click(show_notification_settings)
    recent_notifications_button.on_click(show_recent_notifications)

    menu = widgets.HBox(buttons)
    menu.add_class('par-screen')

    return menu


# Placeholder functions for creating content screens
def create_project_updates_screen():
    content = widgets.HTML("""
    <div class="par-screen">
        <div class="screen-title">Project Updates</div>
        <div class="component">
            <p>Content for project updates goes here.</p>
        </div>
    </div>
    """)
    return content

def create_admin_screen():
    content = widgets.HTML("""
    <div class="par-screen">
        <div class="screen-title">Admin Panel</div>
        <div class="component">
            <p>Content for admin panel goes here.</p>
        </div>
    </div>
    """)
    return content

def create_onshape_search_screen():
    content = widgets.HTML("""
    <div class="par-screen">
        <div class="screen-title">onShape Search</div>
        <div class="component">
            <p>Content for onShape search goes here.</p>
        </div>
    </div>
    """)
    return content

def create_notification_settings_screen():
    content = widgets.HTML("""
    <div class="par-screen">
        <div class="screen-title">Notification Settings</div>
        <div class="component">
            <p>Content for notification settings goes here.</p>
        </div>
    </div>
    """)
    return content

def create_recent_notifications_screen():
    content = widgets.HTML("""
    <div class="par-screen">
        <div class="screen-title">Recent Notifications</div>
        <div class="component">
            <p>Content for recent notifications goes here.</p>
        </div>
    </div>
    """)
    return content



# **RUN CODE**

In [374]:
# Start the application with the main menu
show_main_menu()